Valuing Actions by Estimating Probabilities

In [15]:
from expected_vaep_model.config import chain_file_path
from expected_vaep_model.features.data_preprocessor import ExpVAEPPreprocessor
from expected_vaep_model.features.preprocessing import get_expected_scores, convert_chains_to_schema, create_gamestate_labels, get_stratified_train_test_val_columns
# from expected_vaep_model.features.preprocessing import play_left_to_right, create_duration, get_action_types, get_outcome_types
import expected_score_model.config as config
from expected_vaep_model.modelling_data_contract import ModellingDataContract

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [16]:
chains = pd.read_csv(chain_file_path)
print('Chain data loaded.')

Chain data loaded.


In [17]:
expected_scores_path_dict = {
    'set':{
        'goal':{
            'preprocessor':config.exp_goal_set_preprocessor_file_path,
            'model':config.exp_goal_set_model_file_path},
        'behind':{
            'preprocessor':config.exp_behind_set_preprocessor_file_path,
            'model':config.exp_behind_set_model_file_path},
        'miss':{
            'preprocessor':config.exp_miss_set_preprocessor_file_path,
            'model':config.exp_miss_set_model_file_path}},
    'open':{
        'goal':{
            'preprocessor':config.exp_goal_open_preprocessor_file_path,
            'model':config.exp_goal_open_model_file_path},
        'behind':{
            'preprocessor':config.exp_behind_open_preprocessor_file_path,
            'model':config.exp_behind_open_model_file_path},
        'miss':{
            'preprocessor':config.exp_miss_open_preprocessor_file_path,
            'model':config.exp_miss_open_model_file_path}}
}

print('Getting xScores for chains.')
score_chains = get_expected_scores(chains, expected_scores_path_dict)


Getting xScores for chains.


In [18]:
schema_chains = convert_chains_to_schema(score_chains)
schema_chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type0,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Time_Since_Last_Action,Distance_Since_Last_Action,Chain_Duration,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,xGoals,xBehinds,xMiss,xGoals_normalised,xBehinds_normalised,xMiss_normalised,xScore,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type
0,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,0,0,Centre Bounce,False,True,False,False,False,Hard Ball Get,None,None,8.0,0.0,NaN,NaN,-5.0,0.0,NaN,NaN,24.0,13.0,NaN,NaN,11.0,9.433981,11.0,70.0,5.0,70.178344,0.071307,4.085617,0.091154,5.222739,0.278097,15.933804,4900.0,4.248495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carry,1.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,-6.0,8.0,-5.0,9.0,-6.0,8.0,-5.0,9.0,-6.0,70.178344,69.260378,NaN,1692.0,2121.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,1.0,NaN,effective
1,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,0,0,Hard Ball Get,False,True,False,False,False,Handball,Centre Bounce,None,9.0,8.0,0.0,NaN,-6.0,-5.0,0.0,NaN,24.0,24.0,13.0,NaN,0.0,1.414214,11.0,69.0,6.0,69.260378,0.086738,4.969741,0.092254,5.285792,0.281582,16.133481,4761.0,4.234107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,-7.0,9.0,-6.0,11.0,-7.0,9.0,-6.0,11.0,-7.0,69.260378,67.364679,NaN,1692.0,2121.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,NaN,NaN,ineffective
2,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,0,0,Handball,False,True,False,False,False,Loose Ball Get,Hard Ball Get,Centre Bounce,11.0,9.0,8.0,0.0,-7.0,-6.0,-5.0,0.0,28.0,24.0,24.0,13.0,4.0,2.236068,15.0,67.0,7.0,67.364679,0.104100,5.964487,0.094705,5.426178,0.289349,16.578485,4489.0,4.204693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carry,0.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-5.0,11.0,-7.0,12.0,-5.0,-11.0,7.0,-12.0,5.0,89.274856,90.138782,NaN,1692.0,2121.0,2063.0,1920.0,28.0,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,0.0,NaN,effective
3,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,0,0,Loose Ball Get,False,True,False,False,False,Handball,Handball,Hard Ball Get,12.0,11.0,9.0,8.0,-5.0,-7.0,-6.0,-5.0,29.0,28.0,24.0,24.0,1.0,2.23606

In [19]:
schema_chains.shape

(957696, 115)

In [20]:
print("Creating Expected VAEP Preprocessor.")
preproc = ExpVAEPPreprocessor(expected_scores_path_dict)
preproc.fit(chains)

Creating Expected VAEP Preprocessor.


ExpVAEPPreprocessor(expected_scores_path_dict={'open': {'behind': {'model': '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/expected_behind_open_v8.joblib',
                                                                   'preprocessor': '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/open_behind_preproc.joblib'},...
                                                                'preprocessor': '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/set_goal_preproc.joblib'},
                                                       'miss': {'model': '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/expected_miss_set_v6.joblib',
                                                                'preprocessor': '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/set_miss_preproc.joblib'}}})

In [21]:
print("Creating modelling features.")
exp_vaep_features = preproc.transform(chains)

Creating modelling features.


In [22]:
exp_vaep_features.head()

,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Spoil_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Spoil_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Spoil_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,type_Carry_outcome_effective_a2,type_Carry_outcome_ineffective_a2,type_Carry_outcome_clanger_a2,type_Spoil_outcome_effective_a2,type_Spoil_outcome_ineffective_a2,type_Spoil_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff
0,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,True,True,0.0,0.0,1.0,-1.0,1.414214,1.0,-1.0,1.414214,0,0,0
1,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1,24.0,24.0,9.0,-6.0,11.0,-7.0,2.0,-1.0,2.236068,NaN,NaN,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,True,True,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0,0,0
2,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,28.0,28.0,-11.0,7.0,-12.0,5.0,-1.0,-2.0,2.236068,0.0,NaN,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1,24.0,24.0,9.0,-6.0,11.0,-7.0,2.0,-1.0,2.236068,NaN,NaN,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,1,24.0,24.0,8.0,-5.0,9.0,-6.0,1.0,-1.0,1.414214,1.0,NaN,False,False,-4.0,-4.0,22.0,-14.0,26.076810,20.0,-13.0,23.853721,0,0,0
3,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,Fa

In [23]:
list(exp_vaep_features)

['type_Kick_a0',
 'type_Handball_a0',
 'type_Carry_a0',
 'type_Spoil_a0',
 'outcome_effective_a0',
 'outcome_ineffective_a0',
 'outcome_clanger_a0',
 'type_Kick_outcome_effective_a0',
 'type_Kick_outcome_ineffective_a0',
 'type_Kick_outcome_clanger_a0',
 'type_Handball_outcome_effective_a0',
 'type_Handball_outcome_ineffective_a0',
 'type_Handball_outcome_clanger_a0',
 'type_Carry_outcome_effective_a0',
 'type_Carry_outcome_ineffective_a0',
 'type_Carry_outcome_clanger_a0',
 'type_Spoil_outcome_effective_a0',
 'type_Spoil_outcome_ineffective_a0',
 'type_Spoil_outcome_clanger_a0',
 'quarter_a0',
 'quarter_seconds_a0',
 'overall_seconds_a0',
 'left_right_start_x_a0',
 'left_right_start_y_a0',
 'left_right_end_x_a0',
 'left_right_end_y_a0',
 'dx_a0',
 'dy_a0',
 'movement_a0',
 'contested_a0',
 'mark_a0',
 'type_Kick_a1',
 'type_Handball_a1',
 'type_Carry_a1',
 'type_Spoil_a1',
 'outcome_effective_a1',
 'outcome_ineffective_a1',
 'outcome_clanger_a1',
 'type_Kick_outcome_effective_a1',
 't

In [ ]:
print("Creating labels.")
exp_vaep_labels = create_gamestate_labels(schema_chains)
exp_vaep_modelling_data = pd.concat([schema_chains, exp_vaep_features, exp_vaep_labels], axis=1)

In [ ]:
exp_vaep_modelling_data[exp_vaep_modelling_data['match_id'] == '2023F4_Collingwood_BrisbaneLions'].head(10)